## 84. Funciones para trabajo con collecciones:

In [7]:
%load_ext autoreload
%autoreload 2

import os
path = os.environ.get("HOME")
os.chdir(f"{path}/github/pyspark-hand-on")
from src.utils.utils import spark_session
from pyspark.sql.functions import col, broadcast

spark = spark_session()

25/11/04 19:30:44 WARN Utils: Your hostname, personal-computer resolves to a loopback address: 127.0.1.1; using 172.17.0.1 instead (on interface docker0)
25/11/04 19:30:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/04 19:30:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
df = spark.read.parquet("./data/colecciones/parquet")
df.printSchema()

root
 |-- dia: string (nullable = true)
 |-- tareas: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [9]:
df.show(truncate=False)

+-----+--------------------------------------------+
|dia  |tareas                                      |
+-----+--------------------------------------------+
|lunes|[hacer la tarea, buscar agua, lavar el auto]|
+-----+--------------------------------------------+



In [10]:
from pyspark.sql.functions import col, size, sort_array, array_contains

In [11]:
df.select(
    size(col("tareas")).alias("tamaño"),
    sort_array(col("tareas")).alias("arreglo_ordenado"),
    array_contains(col("tareas"), "buscar agua").alias("buscar_agua")
).show(truncate=False)

+------+--------------------------------------------+-----------+
|tamaño|arreglo_ordenado                            |buscar_agua|
+------+--------------------------------------------+-----------+
|3     |[buscar agua, hacer la tarea, lavar el auto]|true       |
+------+--------------------------------------------+-----------+



#### Función explode:

In [7]:
from pyspark.sql.functions import explode

In [8]:
df.select(
    col("dia"),
    explode(col("tareas")).alias("tareas")
).show()

+-----+--------------+
|  dia|        tareas|
+-----+--------------+
|lunes|hacer la tarea|
|lunes|   buscar agua|
|lunes| lavar el auto|
+-----+--------------+



#### Formato Json:

In [12]:
json_df_str = spark.read.parquet("./data/colecciones/JSON")

In [13]:
json_df_str.show(truncate=False)

+---------------------------------------------------------------------------+
|tareas_str                                                                 |
+---------------------------------------------------------------------------+
|{"dia": "lunes","tareas": ["hacer la tarea","buscar agua","lavar el auto"]}|
+---------------------------------------------------------------------------+



In [14]:
json_df_str.printSchema()

root
 |-- tareas_str: string (nullable = true)



In [15]:
from pyspark.sql.types import StructType, StructField, StringType, ArrayType

In [16]:
schema_json = StructType(
    [
        StructField("dia", StringType(), True),
        StructField("tareas", ArrayType(StringType()), True)
    ]
)

In [22]:
from pyspark.sql.functions import from_json, to_json

In [23]:
json_df = json_df_str.select(
    from_json(col("tareas_str"), schema_json).alias("por_hacer")
)

In [24]:
json_df.printSchema()

root
 |-- por_hacer: struct (nullable = true)
 |    |-- dia: string (nullable = true)
 |    |-- tareas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)



In [26]:
json_df.select(
    col("por_hacer").getItem("dia"),
    col("por_hacer").getItem("tareas"),
    col("por_hacer").getItem("tareas").getItem(0).alias("primer_tarea"),
).show(truncate=False)

+-------------+--------------------------------------------+--------------+
|por_hacer.dia|por_hacer.tareas                            |primer_tarea  |
+-------------+--------------------------------------------+--------------+
|lunes        |[hacer la tarea, buscar agua, lavar el auto]|hacer la tarea|
+-------------+--------------------------------------------+--------------+

